In [32]:
import functools
from dataclasses import dataclass
import random
import numpy as np
from tqdm.auto import tqdm
from icecream import ic
import pandas as pd
from itertools import permutations

In [ ]:
COSTS =[]

In [33]:
data = pd.read_csv('cities/vanuatu.csv', header=None , names=['City', 'x', 'y'])
# cities #to print the data of all the cities
cities = data[['x', 'y']].values
city_name = data['City'].values

In [26]:
# for c in cities.iterrows():
#     print()

In [27]:
# # prof way of calculating the distance matrix
# dist_matrix = np.zeros((len(cities), len(cities)))
# for a, b in product(cities.iterrows(), repeat =2):
#     dist_matrix[a[0], b[0]] = ((a[1]['x'])**2 + (a[1]['y'])**2)**.5
#     city_names = np.array([c['name'] for _, c in cities.iterrows()])
# city_names 

In [ ]:
# prof set of solution
# ic(dist_matrix[0])
# city =0
# ic(city_name[city])
# closest_city = np.argmin(dist_matrix[0])
# ic(closest_city, city_names[closest_city])

In [36]:
# the distance can be an euclidean distance
def distance(city1, city2):
    return np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)
#to calculate total tour distance
def tot_distance(tour, cities):
    dist = 0
    for i in range(len(tour)):
        dist += distance(cities[tour[i]], cities[tour[(i+1)%len(tour)]])
    return dist


In [37]:
#Brute forse algorithm
#tries every possible permutaion of the cities to find the shortest path. 
# Slow but accurate algoritm
def tsp_bruteforce(cities, city_names):
    n = len(cities)
    min_cost = float('inf')
    best_tour = None
    
    # Generate all permutations of city indices
    for perm in permutations(range(n)):
        # Calculate the total cost of the current permutation (tour)
        cost = 0
        for i in range(n - 1):
            cost += distance(cities[perm[i]], cities[perm[i + 1]])
        cost += distance(cities[perm[-1]], cities[perm[0]])  # Return to the origin
        
        # Check if this tour is better
        if cost < min_cost:
            min_cost = cost
            best_tour = perm
    
    # Convert best_tour indices to city names
    best_tour_names = [city_names[i] for i in best_tour]
    
    return best_tour, best_tour_names, min_cost


In [38]:
best_tour_indices, best_tour_names, min_cost = tsp_bruteforce(cities, city_names)

print(f"Brute force TSP cost: {min_cost}")
print(f"Best tour (indices): {best_tour_indices}")
print(f"Best tour (names): {best_tour_names}")

Brute force TSP cost: 12.22271518168697, Best tour: (4, 1, 7, 0, 2, 6, 5, 3)


In [30]:

#try to find using some greddy algorithm
def create_population(pop_size, num_cities):
    return [random.sample(range(num_cities), num_cities) for _ in range(pop_size)]

# #fitness function
def fitness(tour, cities):
    return 1 / tot_distance(tour, cities)

# Crossover (Order Crossover)
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    
    child = [-1] * size
    child[start:end] = parent1[start:end]
    
    for i in range(size):
        if parent2[i] not in child:
            for j in range(size):
                if child[j] == -1:
                    child[j] = parent2[i]
                    break
    return child


# Mutation (Swap Mutation)
def mutate(tour, mutation_rate=0.01):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(tour)), 2)
        tour[i], tour[j] = tour[j], tour[i]
    return tour

# GA Loop
def genetic_algorithm(cities, population_size=100, generations=500, mutation_rate=0.01):
    num_cities = len(cities)
    population = create_population(population_size, num_cities)
    
    for generation in range(generations):
        fitnesses = [fitness(individual, cities) for individual in population]
        
        # Selection and breeding
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = selection(population, fitnesses)
            child1 = mutate(crossover(parent1, parent2), mutation_rate)
            child2 = mutate(crossover(parent2, parent1), mutation_rate)
            new_population.extend([child1, child2])
        
        population = new_population
        
        # Track the best solution
        best_fitness = max(fitnesses)
        best_tour = population[fitnesses.index(best_fitness)]
        best_cost = tot_distance(best_tour, cities)
        
        print(f"Generation {generation + 1}: Best cost = {best_cost}")
    
    return best_tour, best_cost






In [ ]:
best_tour_ga, best_cost_ga = genetic_algorithm(cities)
print(f"Genetic Algorithm TSP cost: {best_cost_ga}, Best tour: {best_tour_ga}")

In [ ]:
# TSP (Travelling salesman problem)
